In [23]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
import pandas as pd
from tensorflow import keras
import os

In [24]:
# 使用csv数据进行转化
# 获取文件名
source_dir = './generate_csv'
print(os.listdir(source_dir))

def get_filenames_by_prefix(source_dir, prefix_name):
    all_files = os.listdir(source_dir)
    results = []
    for filename in all_files:
        if filename.startswith(prefix_name):
            results.append(os.path.join(source_dir, filename))
    return results

train_filenames = get_filenames_by_prefix(source_dir, 'train')
valid_filenames = get_filenames_by_prefix(source_dir, 'valid')
test_filenames = get_filenames_by_prefix(source_dir, 'valid')

import pprint
pprint.pprint(train_filenames)
pprint.pprint(valid_filenames)
pprint.pprint(test_filenames)

['train_15.csv', 'train_01.csv', 'train_00.csv', 'train_14.csv', 'test_08.csv', 'train_02.csv', 'train_16.csv', 'train_17.csv', 'train_03.csv', 'test_09.csv', 'train_07.csv', 'train_13.csv', 'train_12.csv', 'train_06.csv', 'train_10.csv', 'train_04.csv', 'train_05.csv', 'train_11.csv', 'valid_01.csv', 'valid_00.csv', 'valid_02.csv', 'valid_03.csv', 'valid_07.csv', 'valid_06.csv', 'valid_04.csv', 'valid_05.csv', 'valid_08.csv', 'valid_09.csv', 'test_02.csv', 'train_08.csv', 'train_09.csv', 'test_03.csv', 'test_01.csv', 'test_00.csv', 'test_04.csv', 'test_05.csv', 'test_07.csv', 'train_19.csv', 'train_18.csv', 'test_06.csv']
['./generate_csv/train_15.csv',
 './generate_csv/train_01.csv',
 './generate_csv/train_00.csv',
 './generate_csv/train_14.csv',
 './generate_csv/train_02.csv',
 './generate_csv/train_16.csv',
 './generate_csv/train_17.csv',
 './generate_csv/train_03.csv',
 './generate_csv/train_07.csv',
 './generate_csv/train_13.csv',
 './generate_csv/train_12.csv',
 './generate_csv/

In [25]:
# 读取csv文件的代码 从之前的代码中复制过来的
def parse_csv_line(line, n_fields=9):
    defs = [tf.constant(np.nan)] * n_fields # 解析出来是float32类型
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(parsed_fields[0:-1])
    y = tf.stack(parsed_fields[-1:])
    return x, y

def csv_reader_dataset(filenames, n_readers=5, batch_size=32, n_parse_threads=5, shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(lambda filename: tf.data.TextLineDataset(filename).skip(1), cycle_length=n_readers)
    dataset.shuffle(shuffle_buffer_size)    # 打乱数据
    dataset = dataset.map(parse_csv_line, num_parallel_calls=n_parse_threads) # 类似于interleave
    dataset = dataset.batch(batch_size)

    return dataset

batch_size = 32
train_set = csv_reader_dataset(train_filenames, batch_size=batch_size)
valid_set = csv_reader_dataset(valid_filenames, batch_size=batch_size)
test_set = csv_reader_dataset(test_filenames, batch_size=batch_size)

In [26]:
# 遍历dataset，并将数据写入tfrecord
def serialize_example(x, y):
    # converts x, y to tf.train.Example and serialize
    input_features = tf.train.FloatList(value = x)
    label = tf.train.FloatList(value = y)
    features = tf.train.Features(
        feature = {
            'input_features': tf.train.Feature(float_list=input_features),
            'label': tf.train.Feature(float_list=label)
        }
    )
    example = tf.train.Example(features=features)
    return example.SerializePartialToString()

def csv_dataset_to_tfrecords(base_filename, dataset, n_shards, steps_per_shard, compression_type=None):  #n_shards：存成多少个文件，steps_per_shared:每一个小文件要存成多少步
    options = tf.io.TFRecordOptions(compression_type=compression_type)
    all_filenames = []
    for shard_id in range(n_shards):
        filepath = '{}_{:05d}-of-{:05d}'.format(base_filename, shard_id, n_shards)
        with tf.io.TFRecordWriter(filepath, options) as writer:
            for x_batch, y_batch in dataset.take(steps_per_shard):
                for x_example, y_example in zip(x_batch, y_batch):
                    writer.write(serialize_example(x_example, y_example))
        all_filenames.append(filepath)
    return all_filenames

In [ ]:
# 调用函数写入tfrecords文件 不压缩
n_shards = 20
train_steps_per_shard = 11610 // batch_size // n_shards
valid_steps_per_shard = 3880 // batch_size // n_shards
test_steps_per_shard = 5170 // batch_size // n_shards

output_dir = 'generate_tfrecords'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
train_basename = os.path.join(output_dir, 'train')
valid_basename = os.path.join(output_dir, 'valid')
test_basename = os.path.join(output_dir, 'test')

train_tfrecord_filenames = csv_dataset_to_tfrecords(train_basename, train_set, n_shards, train_steps_per_shard, None)
valid_tfrecord_filenames = csv_dataset_to_tfrecords(valid_basename, valid_set, n_shards, valid_steps_per_shard, None)
test_tfrecord_filenames = csv_dataset_to_tfrecords(test_basename, test_set, n_shards, test_steps_per_shard, None)

In [27]:
# 调用函数写入tfrecords文件 压缩
n_shards = 20
train_steps_per_shard = 11610 // batch_size // n_shards
valid_steps_per_shard = 3880 // batch_size // n_shards
test_steps_per_shard = 5170 // batch_size // n_shards

output_dir = 'generate_tfrecords_zip'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
train_basename = os.path.join(output_dir, 'train')
valid_basename = os.path.join(output_dir, 'valid')
test_basename = os.path.join(output_dir, 'test')

train_tfrecord_filenames = csv_dataset_to_tfrecords(train_basename, train_set, n_shards, train_steps_per_shard, 'GZIP')
valid_tfrecord_filenames = csv_dataset_to_tfrecords(valid_basename, valid_set, n_shards, valid_steps_per_shard, 'GZIP')
test_tfrecord_filenames = csv_dataset_to_tfrecords(test_basename, test_set, n_shards, test_steps_per_shard, 'GZIP')

In [34]:
# 读取文件 for test
expected_features = {
    'input_features': tf.io.FixedLenFeature([8], dtype=tf.float32),
    'label': tf.io.FixedLenFeature([1], dtype=tf.float32)
}

def parse_example(serialized_example):
    example = tf.io.parse_single_example(serialized_example, expected_features)
    return example['input_features'], example['label']

def tfrecords_reader_dataset(filenames, n_readers=5, batch_size=32, n_parse_threads=5, shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(lambda filename: tf.data.TFRecordDataset(filename, compression_type='GZIP'), cycle_length=n_readers)
    dataset.shuffle(shuffle_buffer_size)    # 打乱数据
    dataset = dataset.map(parse_example, num_parallel_calls=n_parse_threads) # 类似于interleave
    dataset = dataset.batch(batch_size)

    return dataset

tfrecords_train = tfrecords_reader_dataset(train_tfrecord_filenames, batch_size=3)
for x_batch, y_batch in tfrecords_train.take(2):
    print(x_batch)
    print(y_batch)


tf.Tensor(
[[-1.1157656   0.99306357 -0.334192   -0.06535219 -0.32893205  0.04343066
  -0.12785879  0.30707204]
 [ 2.5150437   1.0731637   0.5574401  -0.17273512 -0.6129126  -0.01909157
  -0.5710993  -0.02749031]
 [-0.82195884  1.8741661   0.1821235  -0.03170019 -0.6011179  -0.14337493
   1.0852206  -0.8613995 ]], shape=(3, 8), dtype=float32)
tf.Tensor(
[[0.524  ]
 [5.00001]
 [1.054  ]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[ 2.5150437   1.0731637   0.5574401  -0.17273512 -0.6129126  -0.01909157
  -0.5710993  -0.02749031]
 [-0.66722274 -0.04823952  0.34529406  0.53826684  1.8521839  -0.06112538
  -0.8417093   1.5204847 ]
 [ 0.8015443   0.27216142 -0.11624393 -0.20231152 -0.5430516  -0.02103962
  -0.5897621  -0.08241846]], shape=(3, 8), dtype=float32)
tf.Tensor(
[[5.00001]
 [1.59   ]
 [3.226  ]], shape=(3, 1), dtype=float32)


In [35]:
# 读取要训练的数据
batch_size = 32
tfrecords_train_set = tfrecords_reader_dataset(train_tfrecord_filenames, batch_size=batch_size)
tfrecords_valid_set = tfrecords_reader_dataset(valid_tfrecord_filenames, batch_size=batch_size)
tfrecords_test_set = tfrecords_reader_dataset(test_tfrecord_filenames, batch_size=batch_size)

In [36]:
# 构建模型并训练
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=[8]),
    keras.layers.Dense(1)
])

model.compile(loss='mean_squared_error', optimizer='adam')
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)]

history = model.fit(tfrecords_train_set, validation_data=tfrecords_valid_set, steps_per_epoch=11160 // batch_size, validation_steps = 3870 // batch_size, epochs=100, callbacks=callbacks)


Train for 348 steps, validate for 120 steps
Epoch 1/100
348/348 [==============================] - 2s 6ms/step - loss: 1.8233 - val_loss: 0.9485
Epoch 2/100
348/348 [==============================] - 1s 2ms/step - loss: 0.6053 - val_loss: 0.6628
Epoch 3/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4829 - val_loss: 0.5169
Epoch 4/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4129 - val_loss: 0.4456
Epoch 5/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3898 - val_loss: 0.4198
Epoch 6/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3681 - val_loss: 0.4125
Epoch 7/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3627 - val_loss: 0.4032
Epoch 8/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3552 - val_loss: 0.3961
Epoch 9/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3430 - val_loss: 0.3939
Epoch 10/100
348/348 [=========================

In [37]:
# 测试
model.evaluate(tfrecords_test_set, steps=5160 // batch_size)

161/161 [==============================] - 0s 2ms/step - loss: 0.3610


0.36095897186987147